## ChAMP analysis Pipeline
### Description
This notebook fetches data from GEO database via FTP server and `GEOparse`. The critical phenotypes are identified and a sample sheet suitable for ChAMP analysis is generated.

### Example
GEO_ACCESSION = "GSE199057"

In [6]:
import GEOparse
import os
import subprocess
import gzip

In [7]:
GEO_ACCESSION = input("Enter the GEO accession number: ")
directory = "idats"  # directory for organizing idats

In [ ]:
command = [
    "aria/aria2c",
    f"ftp://ftp.ncbi.nlm.nih.gov/geo/series/{GEO_ACCESSION[:-3]}nnn/{GEO_ACCESSION}/suppl/{GEO_ACCESSION}_RAW.tar",
]

process = subprocess.Popen(
    command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True
)

while True:
    output = process.stdout.readline()
    if output == "" and process.poll() is not None:
        break
    if output:
        print(output.strip())

### Unzip files

In [ ]:
if not os.path.exists(directory):
    os.makedirs(directory)

subprocess.run(["tar", "-xvf", f"{GEO_ACCESSION}_RAW.tar", "-C", directory])

In [ ]:
counter = 0
for filename in os.listdir(directory):
    if filename.endswith(".gz") and filename.startswith("GSM"):
        file_path = os.path.join(directory, filename)
        output_filename = os.path.splitext(file_path)[0]
        with gzip.open(file_path, "rb") as f_in:
            with open(output_filename, "wb") as f_out:
                f_out.write(f_in.read())

        print(f"Unzipped {filename} to {output_filename}")
        counter += 1
print(f"Unzipped {counter} files")

### Set up file mapping

In [ ]:
file_mapping = {}
counter = 0
for f in os.listdir(directory):
    if f.endswith(".idat"):
        print("idat file found: ", f)
        file_mapping[f.split("_")[0]] = f
        counter += 1

print(f"Found {counter} idat files")

### Fetch metadata

In [ ]:
gse = GEOparse.get_GEO(geo=GEO_ACCESSION, destdir="./downloads")

### Inspect a single sample


In [ ]:
for gsm_name, gsm in gse.gsms.items():
    print("Name: ", gsm_name)
    print("Metadata:")
    for key, value in gsm.metadata.items():
        print(" - %s : %s" % (key, ", ".join(value)))
    print("Table data:")
    print(gsm.table.head())
    break

### Identify phenotype

In [ ]:
import pandas as pd

df = pd.DataFrame()
for i in range(len(gse.gsms)):
    sample_list = gse.gsms[list(gse.gsms.keys())[i]].metadata["characteristics_ch1"]
    sample_dict = {}
    for sample in sample_list:
        sample_dict[sample.split(": ")[0]] = sample.split(": ")[1]
    df = pd.concat([df, pd.DataFrame(sample_dict, index=[i])])
    
print("feature in characteristics_ch1:")
features = []
for column in df.columns:
    features.append(column)
    print(f"{column}: {df[column].unique()}")

### Generating SampleSheet

In [12]:
if len(features) != 1:
    feature_identifier = features[int(input("Enter the index of the feature you want to use: "))]
    print(f"Using '{feature_identifier}' as feature identifier")
else:
    feature_identifier = features[0]


In [ ]:
sample_sheet = pd.DataFrame()
counter = 0
for _, gsm in gse.gsms.items():

    gsm_name = gsm.name
    basket = {"Sample_Name": "", "Sample_Plate": "", "Pool_ID": ""}

    char_dict = {
        key: value
        for item in gsm.metadata["characteristics_ch1"]
        for key, value in [item.split(": ")]
    }

    basket["Sample_Group"] = char_dict.get(feature_identifier, "").replace(" ", "_")

    if gsm_name in file_mapping:

        file_parts = file_mapping[gsm_name].split("_")
        if len(file_parts) > 2:
            basket["Sentrix_ID"] = file_parts[1]
            basket["Sentrix_Position"] = file_parts[2]

        for _ in range(2):
            sample_sheet = pd.concat(
                [sample_sheet, pd.DataFrame([basket])], ignore_index=True
            )

        counter += 1
print(f"matched {counter} samples ({counter*2} entries in sample sheet)")

### Export SampleSheet and organize idats files

In [96]:
if not os.path.exists(f"{GEO_ACCESSION}"):
    os.makedirs(f"{GEO_ACCESSION}")

In [97]:
sample_sheet.sort_values(
    by=["Sample_Group", "Sentrix_ID", "Sentrix_Position"], inplace=True
)
sample_sheet.reset_index(drop=True, inplace=True)
sample_sheet["Sample_Name"] = sample_sheet.index + 1
sample_sheet.to_csv(f"{GEO_ACCESSION}/sample_sheet.csv", index=False)

In [ ]:
counter = 0
for files in os.listdir(directory):
    if files.endswith(".idat"):
        new_file_name = "_".join(files.split("_")[1:])
        print(f"Moving {files} to {GEO_ACCESSION}/{new_file_name}")
        os.rename(f"{directory}/{files}", f"{GEO_ACCESSION}/{new_file_name}")
        counter += 1

print(f"Moved {counter} files")

### REMOVE GEO_ACCESSION TAR AND FILES in DIRECTORY (OPTIONAL)
make sure all idat file is properly extracted before running this cell

In [ ]:
os.remove(f"{GEO_ACCESSION}_RAW.tar")

In [ ]:
counter = 0
for filename in os.listdir(directory):
    os.remove(os.path.join(directory, filename))
    counter += 1
print(f"Removed {counter} files")